# 📘 Prompt Engineering Course Activities — Solutions

This notebook contains reference solutions for the activities in `course-activities.ipynb`.

- Run the setup cell first.
- Use these solutions after attempting the hint-only activities.
- For learning, compare your prompts with these references and note differences.


In [7]:
# Setup code (same as activities)
import warnings
warnings.filterwarnings('ignore')

import openai
import traceback
import requests
import base64
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Open AI version to use
openai.api_type = "azure"
openai.api_version = "2024-12-01-preview"

# Get API_KEY wrapped in token - using environment variables
client_id = os.getenv('CISCO_CLIENT_ID')
client_secret = os.getenv('CISCO_CLIENT_SECRET')

url = "https://id.cisco.com/oauth2/default/v1/token"

payload = "grant_type=client_credentials"
value = base64.b64encode(f"{client_id}:{client_secret}".encode("utf-8")).decode("utf-8")
headers = {
    "Accept": "*/*",
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {value}",
}

token_response = requests.request("POST", url, headers=headers, data=payload)
token_data = token_response.json()

from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint="https://chat-ai.cisco.com",
    api_key=token_data.get('access_token'),
    api_version="2024-12-01-preview"
)

app_key = os.getenv("CISCO_OPENAI_APP_KEY")

def get_chat_completion(messages, model="gpt-4o", temperature=0.0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        user=f'{{"appkey": "{app_key}"}}'
    )
    return response.choices[0].message.content

print("✅ Environment setup complete! Ready for solutions.")


✅ Environment setup complete! Ready for solutions.


In [1]:
# Solution: Activity 1.2 Task 1 - Identify missing elements
print("📝 SOLUTION 1.2 - MISSING ELEMENTS:")
print("\nPrompt 1 has:")
print("✓ Input Data (the code)")
print("✗ Missing: Clear Instructions, Context, Output Format")

print("\nPrompt 2 has:")
print("✓ Instructions/Persona")
print("✗ Missing: Input Data, Context, Output Format")

print("\nPrompt 3 has:")
print("✓ Instructions, Output Format")
print("✗ Missing: Input Data, Context")
print("\n" + "="*80)


📝 SOLUTION 1.2 - MISSING ELEMENTS:

Prompt 1 has:
✓ Input Data (the code)
✗ Missing: Clear Instructions, Context, Output Format

Prompt 2 has:
✓ Instructions/Persona
✗ Missing: Input Data, Context, Output Format

Prompt 3 has:
✓ Instructions, Output Format
✗ Missing: Input Data, Context



In [5]:
# Solution: Activity 1.2 Task 2 - Complete 4-element prompt
system_message = """You are an expert technical writer specializing in Python documentation.
Your task is to create comprehensive docstrings following Google style guide."""

user_message = """Context: This function is part of a financial application that processes user transactions.
It needs clear documentation for both developers and auditors.

Function to document:
def process_transaction(user_id, amount, transaction_type):
    if transaction_type not in ['deposit', 'withdrawal']:
        raise ValueError("Invalid transaction type")
    
    if amount <= 0:
        raise ValueError("Amount must be positive")
    
    balance = get_user_balance(user_id)
    
    if transaction_type == 'withdrawal' and balance < amount:
        raise InsufficientFundsError("Insufficient funds")
    
    new_balance = balance + amount if transaction_type == 'deposit' else balance - amount
    update_user_balance(user_id, new_balance)
    log_transaction(user_id, amount, transaction_type)
    
    return new_balance

Please provide:
1. A comprehensive docstring including:
   - Brief description
   - Args section with parameter types and descriptions
   - Returns section
   - Raises section for all possible exceptions
   - Example usage
2. Inline comments for complex logic
3. Type hints for the function signature"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

response = get_chat_completion(messages)
print("✅ SOLUTION 1.2 - COMPLETE PROMPT WITH ALL 4 ELEMENTS:")
print(response)
print("\n" + "="*80)


✅ SOLUTION 1.2 - COMPLETE PROMPT WITH ALL 4 ELEMENTS:
```python
def process_transaction(user_id: int, amount: float, transaction_type: str) -> float:
    """Processes a financial transaction for a user.

    This function handles user transactions by either depositing or withdrawing
    a specified amount from the user's account. It ensures that the transaction
    type is valid and that the user has sufficient funds for withdrawals.

    Args:
        user_id (int): The unique identifier of the user.
        amount (float): The amount of money to be processed. Must be positive.
        transaction_type (str): The type of transaction, either 'deposit' or 'withdrawal'.

    Returns:
        float: The new balance of the user's account after the transaction.

    Raises:
        ValueError: If the transaction type is invalid or if the amount is not positive.
        InsufficientFundsError: If a withdrawal is attempted with insufficient funds.

    Example:
        >>> process_transaction

In [6]:
# Solution: Activity 2.1 - Convert vague to specific
vague_prompt = "Fix this function"
function_with_issues = """
def calc_price(items, tax, discount):
    total = 0
    for i in items:
        total = total + i
    return total + tax - discount
"""

specific_prompt = """
Refactor this pricing calculation function with the following requirements:
1. Add input validation for all parameters
2. Add type hints following Python typing standards
3. Handle edge cases (empty lists, None values, negative numbers)
4. Improve variable names for clarity
5. Add a docstring explaining the function's purpose
6. Ensure the function handles decimal precision correctly for currency
7. Make the tax calculation percentage-based rather than a fixed amount
"""

print("🔴 VAGUE PROMPT RESULT:")
messages = [
    {"role": "user", "content": f"{vague_prompt}:\n\n```python\n{function_with_issues}\n```"}
]
vague_result = get_chat_completion(messages)
print(vague_result)
print("\n" + "="*80 + "\n")

print("🟢 SPECIFIC PROMPT RESULT:")
messages = [
    {"role": "user", "content": f"{specific_prompt}\n\n```python\n{function_with_issues}\n```"}
]
specific_result = get_chat_completion(messages)
print(specific_result)


🔴 VAGUE PROMPT RESULT:
The function you provided calculates the total price of a list of items, adds tax, and subtracts a discount. However, there are a few improvements and potential issues to consider:

1. **Tax Calculation**: The function currently adds the tax directly to the total. Typically, tax is a percentage of the total price, not a fixed amount. If `tax` is meant to be a percentage, you should calculate it based on the total price.

2. **Discount Calculation**: Similar to tax, if the discount is a percentage, it should be calculated based on the total price.

3. **Variable Naming**: Using more descriptive variable names can improve readability.

Here's a revised version of the function, assuming `tax` and `discount` are percentages:

```python
def calc_price(items, tax_rate, discount_rate):
    subtotal = sum(items)
    tax_amount = subtotal * (tax_rate / 100)
    discount_amount = subtotal * (discount_rate / 100)
    total_price = subtotal + tax_amount - discount_amount
   

In [ ]:
# Solution: Activity 2.2 - Personas
code_to_review = """
def user_login(username, password):
    users = get_all_users()  # Loads entire user database
    for user in users:
        if user['username'] == username and user['password'] == password:
            session_id = generate_random_string(10)
            save_session(session_id, user['id'])
            return {"success": True, "session_id": session_id}
    return {"success": False, "message": "Invalid credentials"}
"""

security_messages = [
    {"role": "system", "content": "You are a Security Engineer reviewing code for security vulnerabilities. Focus on authentication weaknesses, data exposure, and secure coding practices."},
    {"role": "user", "content": f"Review this login function:\n\n```python\n{code_to_review}\n```"}
]
performance_messages = [
    {"role": "system", "content": "You are a Performance Engineer reviewing code for efficiency and scalability issues. Focus on bottlenecks, resource usage, and optimization opportunities."},
    {"role": "user", "content": f"Review this login function:\n\n```python\n{code_to_review}\n```"}
]
qa_messages = [
    {"role": "system", "content": "You are a QA Engineer reviewing code for testing and quality assurance. Focus on edge cases, error handling, and testability."},
    {"role": "user", "content": f"Review this login function:\n\n```python\n{code_to_review}\n```"}
]

security_review = get_chat_completion(security_messages)
performance_review = get_chat_completion(performance_messages)
qa_review = get_chat_completion(qa_messages)

print("🔒 SECURITY ENGINEER PERSPECTIVE:")
print(security_review)
print("\n" + "="*80 + "\n")

print("⚡ PERFORMANCE ENGINEER PERSPECTIVE:")
print(performance_review)
print("\n" + "="*80 + "\n")

print("🧪 QA ENGINEER PERSPECTIVE:")
print(qa_review)


In [ ]:
# Solution: Activity 2.3 - Delimiter mastery
system_message = "You are a software architect. Refactor the provided multi-file code using the organized input sections."

user_message = """
### REFACTORING REQUIREMENTS ###
- Extract shared logic into utility functions
- Improve error handling across all files
- Add proper logging and monitoring
- Follow SOLID principles
### 

### ORIGINAL CODE ###
<file path="models.py">
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    def save(self):
        # Direct database access - not ideal
        db.execute("INSERT INTO users (name, email) VALUES (?, ?)", (self.name, self.email))
</file>

<file path="handlers.py">
def create_user(request):
    name = request.get('name')
    email = request.get('email')
    
    # No validation
    user = User(name, email)
    user.save()
    return {"success": True}

def get_user(user_id):
    # Direct query - no error handling
    result = db.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    return result.fetchone()
</file>
###

### TARGET ARCHITECTURE ###
- Repository pattern for data access
- Service layer for business logic
- Proper dependency injection
- Comprehensive error handling
###

Provide refactored code with clear separation of concerns.
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

response = get_chat_completion(messages)
print("🏗️ SOLUTION - MULTI-FILE REFACTORING:")
print(response)


In [ ]:
# Solution: Activity 2.4 - Step-by-step reasoning
system_message = """
Review the following code using these systematic steps:

Step 1 - Analyze code structure and identify main components
Step 2 - Check for potential bugs and logic errors  
Step 3 - Evaluate performance and efficiency concerns
Step 4 - Assess code maintainability and readability
Step 5 - Provide prioritized recommendations with specific fixes

Follow each step methodically and show your reasoning.
"""

code_to_review = """
def process_orders(orders):
    processed = []
    total_revenue = 0
    
    for order in orders:
        if order['status'] == 'pending':
            # Calculate order total
            item_total = 0
            for item in order['items']:
                item_total += item['price'] * item['quantity']
            
            # Apply discount
            if order['customer_type'] == 'premium':
                item_total = item_total * 0.9
            elif order['customer_type'] == 'regular':
                if item_total > 100:
                    item_total = item_total * 0.95
            
            # Process payment
            if item_total > 0:
                payment_result = charge_customer(order['customer_id'], item_total)
                if payment_result:
                    order['status'] = 'completed'
                    order['total'] = item_total
                    processed.append(order)
                    total_revenue += item_total
                else:
                    order['status'] = 'failed'
    
    return processed, total_revenue
"""

user_message = f"""
Please review this order processing function:

```python
{code_to_review}
```
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

response = get_chat_completion(messages)
print("🔍 SOLUTION - STEP-BY-STEP REVIEW:")
print(response)


## Solution: Activity 4.1 — Command Creation Challenge

Below are reference templates you can use after attempting your own. Treat these as examples to compare against your work and iterate.

### Claude Code — `.claude/commands/code/review.md`
```markdown
---
allowed-tools: Read, Grep, Bash(git diff:*), Bash(git blame:*)
argument-hint: [focus_area] [language]
description: Comprehensive code review with prioritized actions
---

You are a senior software engineer performing a code review for $2 code.

Focus Area: $1

Perform systematic analysis:
1) Overall Assessment — architecture, structure, complexity
2) Quality — standards, readability, potential bugs, error handling
3) Security — input validation, secrets, injection, authz/authn
4) Performance — hot paths, allocations, I/O patterns
5) Maintainability — naming, cohesion, testability, documentation

Output format:
- Summary (3–5 bullets)
- Findings by category (Critical/High/Medium/Low)
- Actionable Next Steps (prioritized)
```

### Claude Code — `.claude/commands/debug/production.md`
```markdown
---
allowed-tools: Read, Bash(log:*), Bash(grep:*), Bash(ps:*)
argument-hint: [severity] [component]
description: Debug production issue and produce executive summary + technical notes
---

You are analyzing a $1 production incident in the $2 component.

Steps:
1) Root Cause Analysis — symptom timeline, triggers, failure chain
2) Impact Assessment — scope, user impact, SLO/SLA
3) Solution Options — quick fix vs long-term fix, trade-offs
4) Risk & Rollout — deployment plan, fallback, observability

Output format:
- Executive Summary (non-technical, 4–6 lines)
- Technical Notes (root cause, evidence, logs/metrics)
- Mitigation Plan (immediate + long-term)
- Verification & Monitoring
```

### GitHub Copilot — `.github/prompts/code-review.md`
```markdown
---
mode: agent
tools: ['githubRepo', 'terminal', 'codeSearch']
description: Perform a comprehensive code review with prioritized actions
---

You are a senior software engineer conducting a code review.

Provide:
1. Overall assessment
2. Detailed findings by: Quality, Security, Performance, Maintainability
3. Prioritized action list (Critical/High/Medium/Low)
```

### GitHub Copilot — `.github/prompts/debug-production.md`
```markdown
---
mode: agent
tools: ['terminal', 'fileSearch', 'codeAnalysis']
description: Debug a production issue; produce executive summary + technical notes
---

You are debugging a production incident. Provide:
- Executive summary (business-focused)
- Root cause analysis (technical)
- Mitigation plan (immediate/long-term)
- Verification steps & monitoring
```

### Iteration guidance
- Start minimal; run the command; adjust focus, allowed tools, and output sections.
- Tune for brevity and actionability; reduce noise, keep headings stable across runs.
- Validate that Critical/High findings always appear first.

Inspiration (don’t copy verbatim): https://github.com/aws-samples/anthropic-on-aws/tree/main/advanced-claude-code-patterns/commands


## Solution: Activity 4.2 — Advanced Command Patterns (Chained Workflows)

Use this reference after attempting your own workflow design. The goal is to chain specialized commands with clear handoffs, consistent outputs, and optional conditional logic.

### Design principles
- Keep each command focused (single responsibility)
- Define a stable output schema for handoffs (JSON in a fenced block)
- Prefer short, action-oriented sections (Summary, Findings, Next Steps)
- Document which command consumes which handoff fields

### Claude Code — Workflow Orchestrator (feature development)
Example file: `.claude/commands/workflows/feature-dev.md`
```markdown
---
allowed-tools: Read, Grep, Bash(git diff:*)
argument-hint: [feature_name] [component]
description: Orchestrate feature dev flow: spec → scaffold → review → tests
---

You are orchestrating a feature workflow: "$1" in "$2".

Steps:
1) Specification — derive concise requirements and constraints
2) Scaffold Plan — list minimal file changes and stubs
3) Review Focus — define critical review criteria for this change
4) Test Plan — enumerate essential test cases (unit/integration)

Output:
- Executive Summary (3–5 bullets)
- Spec (requirements/constraints/acceptance)
- Scaffold Plan (files, functions, TODOs)
- Review Focus (areas, risks)
- Test Plan (cases by priority)

Handoff (JSON):
```json
{
  "feature": "$1",
  "component": "$2",
  "files": [],
  "functions": [],
  "acceptance": [],
  "review_focus": ["security", "performance"],
  "test_plan": []
}
```
```

### Claude Code — Consumer: Scaffold from Handoff
Example file: `.claude/commands/code/scaffold.md`
```markdown
---
allowed-tools: Read, Write, Bash(git:*)
argument-hint: [language]
description: Generate minimal scaffolding from previous workflow handoff
---

You are generating code scaffolding in $1.

Input contains a prior handoff JSON in a fenced block. Parse it strictly and:
- Create only the listed files/functions with minimal stubs
- Insert TODO markers where implementation is needed
- Include top-of-file docstrings linking to acceptance criteria

Output:
- Summary (created/modified)
- File list with brief descriptions
- Next steps for implementers

Expect an input block like:
```json
{"feature":"", "component":"", "files":[], "functions":[], "acceptance":[]}
```
```

### GitHub Copilot — Workflow Template
Example file: `.github/prompts/feature-workflow.md`
```markdown
---
mode: agent
tools: ['githubRepo', 'codeSearch']
description: Orchestrate feature dev workflow; emit JSON handoff for next command
---

You are orchestrating a feature workflow. Provide:
- Spec (requirements → acceptance)
- Scaffold Plan (files/functions)
- Review Focus
- Test Plan

Emit a JSON handoff in a fenced block at the end with keys:
feature, component, files, functions, acceptance, review_focus, test_plan
```

### Example: Handoff → Consumer chaining
- Run `/feature-dev SignIn user-auth` (Claude Code), copy JSON handoff block
- Run `/scaffold python`, paste the JSON under your request
- Implement stubs, then invoke your code-review command with the same handoff

### Conditional logic ideas
- If the component touches auth or payments, auto-include security review focus
- If scaffold adds migrations, include rollback/backup steps in the Test Plan

### Iteration guidance
- Validate outputs across multiple features; stabilize section headers and JSON keys
- Keep handoffs minimal and documented; avoid brittle implicit fields

Inspiration (don’t copy verbatim): https://github.com/aws-samples/anthropic-on-aws/tree/main/advanced-claude-code-patterns/commands
